In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import tensorflow        as tf
import numpy             as np

import pathlib
import random
import data
import os

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing             import image
from tensorflow.keras.models                    import load_model

In [ ]:
dataObj    = data.Data()
classNames = dataObj.classes
numClasses = dataObj.numClasses

In [ ]:
def getBestSavedModel(modelCheckpointPath):
    dirPaths  = sorted(modelCheckpointPath.glob("*"))
    minLoss   = float('inf')
    bestModel = ""
    for dirPath in dirPaths:
        modelname = pathlib.Path(dirPath.name).stem
        loss = float(modelname.split("_")[-1])
        if loss <= minLoss:
            bestModel = dirPath
            minLoss   = loss
    return str(bestModel)

In [ ]:
def predictCNN():
    def getPredictions(model, framePath):
        frame = image.load_img(framePath, 
                               target_size   = (299,299), 
                               interpolation = "lanczos")   # shape: (299, 299, 3): # of dim: 3
        frame_arr   = image.img_to_array(frame)             # pixel values in range [0,255]
        frame_arr   = preprocess_input(frame_arr)           # pixel values in range [-1, 1]
        frame_arr   = np.expand_dims(frame_arr, axis = 0)   # expands shape to: (1, 299, 299, 3): # of dim: 4
        predictions = model.predict(frame_arr)
        predictions = predictions[0]
        return predictions
    
    def plotPredictions(predictions, predictedLabels):
        plt.grid(False)
        plot = plt.bar(range(len(predictions)), predictions, color="#777777")
        plt.xticks(range(len(predictedLabels)), predictedLabels, rotation=20)
        plt.ylim([0, 1])
#         plt.setp(plot.get_xticklabels(), rotation=30, horizontalalignment='right')
        plot[0].set_color('blue')
        plt.show()
    
    
    dataSetType   = "Test"
    rootPath      = pathlib.Path(r"./")
    dataDirectory = rootPath/'Frames'/dataSetType
    framesPaths   = list(sorted(dataDirectory.glob('*/*.jpg')))
    frameCount    = len(framesPaths)
    
    modelCheckpointPath = rootPath/'Callbacks/CNN'/f'{numClasses}'/'ModelCheckpoint'
    savedModelPath = "./Callbacks/CNN/101/ModelCheckpoint/1589239195_CNN_009_4.05.h5" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    savedModel = load_model(savedModelPath)
    
    topk = 5
    numFrames = 20
    for i in range(numFrames):
        randomIndex     = random.randint(0, frameCount - 1)
        randomFramePath = framesPaths[randomIndex]
        
        actualClass     = randomFramePath.parts[-2]
        predictions     = getPredictions(savedModel, randomFramePath)
        topkindecies    = predictions.argsort()[topk*-1:][::-1]
        topkpredictions = predictions[topkindecies]
        
        predictedLabels    = []
        for index in topkindecies:
            predictedLabels.append(classNames[index])
        
        print("Video name: ", "_".join(randomFramePath.stem.split("_")[:-1]))
        print("Actual class: ", actualClass)
        print(f'Predicted class: {classNames[topkindecies[0]]} - {predictions[topkindecies[0]] *100}%')
        plotPredictions(topkpredictions, predictedLabels)
        print()
        
predictCNN()

In [ ]:
def predictRNN():
    def getPredictions(model, sequencePath):
        maxFrameCount = dataObj.getMaxFrameCount()
        sequence      = np.load(sequencePath)
        sequence      = np.pad(sequence, ((0, maxFrameCount - len(sequence)), (0, 0)), 'edge')
        sequence      = np.expand_dims(sequence, axis = 0)
        predictions   = model.predict(sequence)
        predictions   = predictions[0]
        return predictions
    
    def plotPredictions(predictions, predictedLabels):
        plt.grid(False)
        plot = plt.bar(range(len(predictions)), predictions, color="#777777")
        plt.xticks(range(len(predictedLabels)), predictedLabels, rotation=20)
        plt.ylim([0, 1])
        plot[0].set_color('blue')
        plt.show()
    
    dataSetType    = "Test"
    rootPath       = pathlib.Path(r"./")
    dataDirectory  = rootPath/'Sequences'/dataSetType
    sequencesPaths = list(sorted(dataDirectory.glob('*/*.npy')))
    sequenceCount  = len(sequencesPaths)
    
    modelCheckpointPath = rootPath/'Callbacks/RNN'/f'{numClasses}'/'ModelCheckpoint'
    savedModelPath = "./Callbacks/RNN/101/ModelCheckpoint/1589397436_RNN_045_3.92.h5" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    savedModel = load_model(savedModelPath)
    
    topk = 5
    numSequences = 20
    for i in range(numSequences):
        randomIndex        = random.randint(0, sequenceCount - 1)
        randomSequencePath = sequencesPaths[randomIndex]
        
        actualClass        = randomSequencePath.parts[-2]
        predictions        = getPredictions(savedModel, randomSequencePath)
        topkindecies       = predictions.argsort()[topk*-1:][::-1]
        topkpredictions    = predictions[topkindecies]
        
        predictedLabels    = []
        for index in topkindecies:
            predictedLabels.append(classNames[index])
            
        print("Video name: ", "_".join(randomSequencePath.stem.split("_")[:-1]))
        print("Actual class: ", actualClass)
        print(f'Predicted class: {classNames[topkindecies[0]]} - {predictions[topkindecies[0]] *100}%')
        plotPredictions(topkpredictions, predictedLabels)
        print()
        
predictRNN()